In [9]:
import sqlite3

conn = sqlite3.connect("ecommerce.db")


In [15]:
import pandas as pd
import sqlite3

customer_base = pd.read_sql_query("SELECT * FROM customer_base", conn)
customer_base.head()


,customer_unique_id,frequency,monetary,avg_review_score,last_purchase_date,recency_days,is_repeat_customer,is_high_value
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,5.0,2018-05-10 10:56:27,160,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,4.0,2018-05-07 11:11:27,163,0,0
2,0000f46a3911fa3c0805444483337064,1,86.22,3.0,2017-03-10 21:05:03,585,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,4.0,2017-10-12 20:29:41,369,0,0
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,5.0,2017-11-14 19:45:42,336,0,1


In [17]:
customer_base["churn"] = (customer_base["frequency"] == 1).astype(int)


In [19]:
# Handle missing values before ML
customer_base["avg_review_score"] = customer_base["avg_review_score"].fillna(
    customer_base["avg_review_score"].median()
)

customer_base["monetary"] = customer_base["monetary"].fillna(0)
customer_base["recency_days"] = customer_base["recency_days"].fillna(
    customer_base["recency_days"].median()
)


In [21]:
features = customer_base[
    ["recency_days", "frequency", "monetary", "avg_review_score"]
]

target = customer_base["churn"]


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

baseline_model = LogisticRegression(max_iter=200)

baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print("Baseline F1 Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Baseline F1 Score: 0.9999731637280949
Confusion Matrix:
 [[  588     0]
 [    1 18631]]


In [25]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=100, class_weight="balanced", random_state=42
)

rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict(X_test)

print("Random Forest F1:", f1_score(y_test, rf_pred))


Random Forest F1: 1.0


This notebook defines churn labels and trains baseline machine learning models for customer retention analytics.

In [33]:
customer_base.to_sql("customer_base", conn, if_exists="replace", index=False)


96096

In [35]:
import sqlite3

conn = sqlite3.connect("/Users/ananyaravikumar/anaconda_projects/ecommerce-retention-system/ecommerce.db")

cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(cursor.fetchall())


[]


In [37]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("ecommerce.db")

print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn))


            name
0         orders
1      customers
2       payments
3        reviews
4  customer_base


In [39]:
customer_base.head()

,customer_unique_id,frequency,monetary,avg_review_score,last_purchase_date,recency_days,is_repeat_customer,is_high_value,churn
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,5.0,2018-05-10 10:56:27,160,0,0,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,4.0,2018-05-07 11:11:27,163,0,0,1
2,0000f46a3911fa3c0805444483337064,1,86.22,3.0,2017-03-10 21:05:03,585,0,0,1
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,4.0,2017-10-12 20:29:41,369,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,5.0,2017-11-14 19:45:42,336,0,1,1
